In [1]:
import findspark
findspark.init('C:/Users/gaborstefanics/Documents/GitHub/learnPySark/spark/spark-2.4.4-bin-hadoop2.7')

#import SparkSession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('random_forest').getOrCreate()

In [2]:
#read the dataset
df=spark.read.csv('affairs.csv',inferSchema=True,header=True)

In [3]:
#check the shape of the data 
print((df.count(),len(df.columns)))

(6366, 6)


In [4]:
#printSchema
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)



In [7]:
#view the dataset
df.show(5)

+-------------+----+-----------+--------+---------+-------+
|rate_marriage| age|yrs_married|children|religious|affairs|
+-------------+----+-----------+--------+---------+-------+
|            5|32.0|        6.0|     1.0|        3|      0|
|            4|22.0|        2.5|     0.0|        2|      0|
|            3|32.0|        9.0|     3.0|        3|      1|
|            3|27.0|       13.0|     3.0|        1|      1|
|            4|22.0|        2.5|     0.0|        1|      1|
+-------------+----+-----------+--------+---------+-------+
only showing top 5 rows



In [8]:
#Exploratory Data Analysis
df.describe().select('summary','rate_marriage','age','yrs_married','children','religious').show()

+-------+------------------+------------------+-----------------+------------------+------------------+
|summary|     rate_marriage|               age|      yrs_married|          children|         religious|
+-------+------------------+------------------+-----------------+------------------+------------------+
|  count|              6366|              6366|             6366|              6366|              6366|
|   mean| 4.109644989004084|29.082862079798932| 9.00942507068803|1.3968740182218033|2.4261702796104303|
| stddev|0.9614295945655025| 6.847881883668817|7.280119972766412| 1.433470828560344|0.8783688402641785|
|    min|                 1|              17.5|              0.5|               0.0|                 1|
|    max|                 5|              42.0|             23.0|               5.5|                 4|
+-------+------------------+------------------+-----------------+------------------+------------------+



In [9]:
df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 2053|
|      0| 4313|
+-------+-----+



In [10]:
df.groupBy('rate_marriage').count().orderBy('rate_marriage',ascending=False).show()

+-------------+-----+
|rate_marriage|count|
+-------------+-----+
|            5| 2684|
|            4| 2242|
|            3|  993|
|            2|  348|
|            1|   99|
+-------------+-----+



In [11]:
df.groupBy('rate_marriage','affairs').count().orderBy('rate_marriage','affairs','count',ascending=True).show()

+-------------+-------+-----+
|rate_marriage|affairs|count|
+-------------+-------+-----+
|            1|      0|   25|
|            1|      1|   74|
|            2|      0|  127|
|            2|      1|  221|
|            3|      0|  446|
|            3|      1|  547|
|            4|      0| 1518|
|            4|      1|  724|
|            5|      0| 2197|
|            5|      1|  487|
+-------------+-------+-----+



In [12]:
df.groupBy('religious','affairs').count().orderBy('religious','affairs','count',ascending=True).show()

+---------+-------+-----+
|religious|affairs|count|
+---------+-------+-----+
|        1|      0|  613|
|        1|      1|  408|
|        2|      0| 1448|
|        2|      1|  819|
|        3|      0| 1715|
|        3|      1|  707|
|        4|      0|  537|
|        4|      1|  119|
+---------+-------+-----+



In [13]:
df.groupBy('children','affairs').count().orderBy('children','affairs','count',ascending=True).show()

+--------+-------+-----+
|children|affairs|count|
+--------+-------+-----+
|     0.0|      0| 1912|
|     0.0|      1|  502|
|     1.0|      0|  747|
|     1.0|      1|  412|
|     2.0|      0|  873|
|     2.0|      1|  608|
|     3.0|      0|  460|
|     3.0|      1|  321|
|     4.0|      0|  197|
|     4.0|      1|  131|
|     5.5|      0|  124|
|     5.5|      1|   79|
+--------+-------+-----+



In [14]:
df.groupBy('affairs').mean().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------+
|affairs|avg(rate_marriage)|          avg(age)|  avg(yrs_married)|     avg(children)|    avg(religious)|avg(affairs)|
+-------+------------------+------------------+------------------+------------------+------------------+------------+
|      1|3.6473453482708234|30.537018996590355|11.152459814905017|1.7289332683877252| 2.261568436434486|         1.0|
|      0| 4.329700904242986| 28.39067934152562| 7.989334569904939|1.2388128912589844|2.5045212149316023|         0.0|
+-------+------------------+------------------+------------------+------------------+------------------+------------+



In [19]:
import pandas as pd
pd_df = df.groupBy('affairs').mean().toPandas()
pd_df.head()

,affairs,avg(rate_marriage),avg(age),avg(yrs_married),avg(children),avg(religious),avg(affairs)
0,1,3.647345,30.537019,11.152460,1.728933,2.261568,1.0
1,0,4.329701,28.390679,7.989335,1.238813,2.504521,0.0


In [20]:
from pyspark.ml.feature import VectorAssembler

In [21]:
df_assembler = VectorAssembler(inputCols=['rate_marriage', 'age', 'yrs_married', 'children', 'religious'], outputCol="features")
df = df_assembler.transform(df)

In [22]:
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)
 |-- features: vector (nullable = true)



In [23]:
df.select(['features','affairs']).show(10,False)

+-----------------------+-------+
|features               |affairs|
+-----------------------+-------+
|[5.0,32.0,6.0,1.0,3.0] |0      |
|[4.0,22.0,2.5,0.0,2.0] |0      |
|[3.0,32.0,9.0,3.0,3.0] |1      |
|[3.0,27.0,13.0,3.0,1.0]|1      |
|[4.0,22.0,2.5,0.0,1.0] |1      |
|[4.0,37.0,16.5,4.0,3.0]|1      |
|[5.0,27.0,9.0,1.0,1.0] |1      |
|[4.0,27.0,9.0,0.0,2.0] |1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
+-----------------------+-------+
only showing top 10 rows



In [24]:
#select data for building model
model_df=df.select(['features','affairs'])

In [25]:
train_df,test_df=model_df.randomSplit([0.75,0.25])

In [26]:
train_df.count()

4715

In [27]:
train_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 1517|
|      0| 3198|
+-------+-----+



In [28]:
test_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1|  536|
|      0| 1115|
+-------+-----+



In [29]:
from pyspark.ml.classification import RandomForestClassifier

In [40]:
rf_classifier=RandomForestClassifier(labelCol='affairs',numTrees=100).fit(train_df)

In [41]:
rf_predictions=rf_classifier.transform(test_df)

In [42]:
rf_predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,17.5,0.5,0.0...|      0|[77.8152051726830...|[0.77815205172683...|       0.0|
|[1.0,22.0,2.5,0.0...|      1|[40.5607199455726...|[0.40560719945572...|       1.0|
|[1.0,22.0,2.5,1.0...|      1|[46.1960907704918...|[0.46196090770491...|       1.0|
|[1.0,22.0,2.5,1.0...|      1|[46.1960907704918...|[0.46196090770491...|       1.0|
|[1.0,27.0,2.5,0.0...|      0|[39.9626853232732...|[0.39962685323273...|       1.0|
|[1.0,27.0,2.5,0.0...|      1|[44.1733531683560...|[0.44173353168356...|       1.0|
|[1.0,27.0,2.5,0.0...|      1|[44.1733531683560...|[0.44173353168356...|       1.0|
|[1.0,27.0,2.5,1.0...|      1|[48.9742490641573...|[0.48974249064157...|       1.0|
|[1.0,27.0,6.0,1.0...|      1|[39.1739051279595...|[0.39173905127959...|    

In [43]:
rf_predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 1356|
|       1.0|  295|
+----------+-----+



In [44]:
rf_predictions.select(['probability','affairs','prediction']).show(10,False)

+----------------------------------------+-------+----------+
|probability                             |affairs|prediction|
+----------------------------------------+-------+----------+
|[0.7781520517268308,0.2218479482731692] |0      |0.0       |
|[0.4056071994557265,0.5943928005442735] |1      |1.0       |
|[0.46196090770491843,0.5380390922950816]|1      |1.0       |
|[0.46196090770491843,0.5380390922950816]|1      |1.0       |
|[0.3996268532327323,0.6003731467672678] |0      |1.0       |
|[0.4417335316835604,0.5582664683164397] |1      |1.0       |
|[0.4417335316835604,0.5582664683164397] |1      |1.0       |
|[0.48974249064157355,0.5102575093584264]|1      |1.0       |
|[0.39173905127959585,0.608260948720404] |1      |1.0       |
|[0.392295797370516,0.6077042026294841]  |0      |1.0       |
+----------------------------------------+-------+----------+
only showing top 10 rows



In [45]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [46]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [47]:
rf_accuracy=MulticlassClassificationEvaluator(labelCol='affairs',metricName='accuracy').evaluate(rf_predictions)

In [48]:
print('The accuracy of RF on test data is {0:.0%}'.format(rf_accuracy))

The accuracy of RF on test data is 72%


In [49]:
print(rf_accuracy)

0.724409448819


In [50]:
rf_precision=MulticlassClassificationEvaluator(labelCol='affairs',metricName='weightedPrecision').evaluate(rf_predictions)

In [51]:
print('The precision rate on test data is {0:.0%}'.format(rf_precision))

The precision rate on test data is 71%


In [52]:
rf_precision

0.7089254986069594

In [53]:
rf_auc=BinaryClassificationEvaluator(labelCol='affairs').evaluate(rf_predictions)

In [54]:
print(rf_auc)

0.750173181179


In [55]:
# Feature importance

In [56]:
rf_classifier.featureImportances

SparseVector(5, {0: 0.6233, 1: 0.0262, 2: 0.2289, 3: 0.0525, 4: 0.0691})

In [57]:
df.schema["features"].metadata["ml_attr"]["attrs"]

{u'numeric': [{u'idx': 0, u'name': u'rate_marriage'},
  {u'idx': 1, u'name': u'age'},
  {u'idx': 2, u'name': u'yrs_married'},
  {u'idx': 3, u'name': u'children'},
  {u'idx': 4, u'name': u'religious'}]}

In [58]:
# Save the model 

In [66]:
pwd

u'C:\\Users\\gaborstefanics\\Documents\\GitHub\\machine-learning-with-pyspark\\chapter_6_Random_Forests'

In [76]:
import os
path = os.getcwd() + "/"

path = "C:/Users/gaborstefanics/Documents/GitHub/machine-learning-with-pyspark/chapter_6_Random_Forests/"

In [92]:
rf_classifier.save("C://Users//gaborstefanics//Documents//GitHub//machine-learning-with-pyspark//chapter_6_Random_Forests//RF_model")

Py4JJavaError: An error occurred while calling o1650.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:100)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDDFunctions$

In [93]:
from pyspark.ml.classification import RandomForestClassificationModel

In [94]:
rf=RandomForestClassificationModel.load("RF_model")

Py4JJavaError: An error occurred while calling o1697.load.
: java.lang.UnsupportedOperationException: empty collection
	at org.apache.spark.rdd.RDD$$anonfun$first$1.apply(RDD.scala:1380)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.RDD.first(RDD.scala:1377)
	at org.apache.spark.ml.util.DefaultParamsReader$.loadMetadata(ReadWrite.scala:615)
	at org.apache.spark.ml.tree.EnsembleModelReadWrite$.loadImpl(treeModels.scala:427)
	at org.apache.spark.ml.classification.RandomForestClassificationModel$RandomForestClassificationModelReader.load(RandomForestClassifier.scala:316)
	at org.apache.spark.ml.classification.RandomForestClassificationModel$RandomForestClassificationModelReader.load(RandomForestClassifier.scala:306)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at

In [95]:
model_preditions=rf.transform(test_df)

NameError: name 'rf' is not defined

In [60]:
model_preditions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,22.0,2.5,1.0...|      1|[188.676639360932...|[0.37735327872186...|       1.0|
|[1.0,27.0,2.5,0.0...|      1|[195.425833792250...|[0.39085166758450...|       1.0|
|[1.0,27.0,6.0,0.0...|      0|[193.138478579040...|[0.38627695715808...|       1.0|
|[1.0,27.0,6.0,2.0...|      1|[185.424877645536...|[0.37084975529107...|       1.0|
|[1.0,27.0,6.0,3.0...|      0|[164.685852316351...|[0.32937170463270...|       1.0|
|[1.0,27.0,9.0,4.0...|      0|[142.006095001922...|[0.28401219000384...|       1.0|
|[1.0,32.0,13.0,0....|      1|[176.885399312490...|[0.35377079862498...|       1.0|
|[1.0,32.0,13.0,2....|      1|[128.585405941664...|[0.25717081188332...|       1.0|
|[1.0,32.0,13.0,2....|      1|[126.963464206019...|[0.25392692841203...|    